## FAISS

FAISS is a popular tool for efficient similarity search over vector embeddings. It is often used in machine learning, NLP and RAG applications. FAISS implements fast nearest neighbor search to provide a faster output. 

Before learning about Fash Nearest Neighbor Search, there are few more concepts that we need to be aware of. 

### Nearest Neighbor Search (NNS)
Nearest Neighbor Search can be used to retrieve documents relevant to the query. Nearest Neighbor Search looks for vectors closest to the query vector to retrieve the documents. It uses the either of the below distance metrics to retrieve the documents. 

1. Cosine similarity (angle between vectors)
2. Euclidean distance (L2 norm)

There are two types of Nearest Neighbor Search

- Exact Nearest Neighbor Search (ENN)

    This is a brute force search. It compares the query vector with every vector in the database. While this is 100% accurate, it is too slow for millions of embeddings. 

- Approximate Nearest Neighbor Search (ANN)

    ANN uses clever data structures like trees, graphs or clustering to speed things up. It trades a tiny bit of accuracy for huge performance gains. ANN is widely used in RAG applications. 

### Embedding Space
When we generate embeddings (say using OpenAI Sentence Transformers, or CLIP), each piece of text, image or item is mapped to a vector in a high-dimensional space. (384D, 768D, 1536D). Think of it as a cloud of points floating in a space. The distance between points reflects semantic similarity (close = similar, far = different)

There are various distance metrics that are available to calculate the distance between two points in a high dimensional space. 

### Distance Metrics 
Distance Metrics are various approaches that can be followed to find the distance between the vectors in Nearest Neighbor Search. The distance between the vectors determine the similarity between them. Closer the vectors, more similar they are. There are various distance metrics that are used.

1. Euclidean Distance (L2 Norm)

    - Formulae

        ![alt text](euclidean-distance-formulae.jpg "Euclidean Distance")
    
    - Measures the straight line geometric distance between two vectors. 
    - Intuition: "How apart are the points in space?"
    - Use case: Works well when both the magnitude and the direction of the vectors matter. (Ex: Clustering images or sensor data)

2. Manhattan Distance (L1 Norm)

    - Formulae

        ![alt text](manhattan-distance-formulae.jpg "Manhattan Distance")
    
    - Measures distance as if we can only move along the grid lines. Like city blocks in Manhattan.
    - Intuition: More robust to outliers than euclidean distance. (I doubt this)
    - Use case: Sometimes used in sparse embeddings or table data

3. Cosine Similarity / Cosine Distance

    - Formulae

        ![alt text](cosine-similarity-formulae.jpg "Cosine Similarity")

        Cosine Similarity = 1 - sim(x,y)

    - Focuses on the angle between the vectors
    - Intuition: Two vectors pointing in the same direction are similar even though they differ in length. 
    - Use case: Very common in semantic embeddings (text embeddings, document search, RAG memory) since meaning is captured by direction and not length.

4. Dot Product / Inner Product

    - Formulae

        ![alt text](dot-product-formula.jpg "Dot Product")

    - Dot Product is similar to cosine similarity. We can consider cosine similarity to be a normalized form of dot product. 
    - Dot Product considers magnitude in the score while cosine similarity removes it. 
    - We can consider cosine similarity as pure directional closeness. Dot Product is directional closeness weighted by size. 

### Vector Normalization
Vector Normalization means, scaling down a vector such that its own length (magnitude) becomes 1. After normalization, all vectors lie on the same unit sphere in the embedding space. This ensures that no vector dominates purely because it has a larger magnitude. Some embedding models produces vectors of different lengths even for similar meanings. Normalization ensures only direction (semantic meaning) matters. Lets talk about this in detail. 

 We all know, a vector is just a list of numbers representing some objects (like text, or an image). Example:

 - Vector for "doberman" -> [3,4]
 - Vector for "pug" -> [6,8]

Both these vectors are dogs. But, the vector for put is double the value of doberman, which may give a impression that they are not related to each other. Vector normalization removes this difference. 

Vector normalization scales the vectors such that their length (magnitude) becomes 1 without changing its direction. 

The formula for normalizing a vector is Vn = V/Squareroot(Sum of squares of all vectors)

For the above example, the vector for doberman would be (3/SqRoot(sq(3) + sq(4)),4/SqRoot(sq(3) + sq(4))) which would be [0.6,0.8]

If we add the same formula for pug, the normalized vector would be [0.6, 0.8]

### k-means clustering
Clustering is grouping of similar data-points together. k-means is unsupervised machine learning algorithm that partitions a dataset into **k clusters**. Here k is a predefined number of clusters that we define. In FAISS, this is called as **nlist**. 

Each cluster has a **centroid** and data-points that are closest to that centroid compared to other centroids. 

#### How k-means clustering works?
For a given value of k, the algorithm would

1. **Initialize centroids** randomly. It would pick k random points as initial cluster centers. 
2. **Assigns points to clusters**. Each data point goes to the cluster whose centroid is nearest. (using a distance metric, usually Euclidean). Can two points be equidistant to two or more centroids? While this is theoretically possible, practically, this is rare as the distance is computed as a floating point number. In theory if it happens, the algorithm will assign the data point to the first centroid it loads to the memory. 
3. **Recompute the centroids**. After all the data-points are assigned to their random centroids, The algorithm would **compute their mean (average) position** across each dimension. That average becomes the new centroid of the cluster. This is repeated for every cluster. 

#### When is this algorithm stopped?
1. **Centroids don't change anymore (Convergence)**
    If the new centroids are the same (or almost the same) as the old ones, the algorithm has converged. This means the clusters are stable.

2. **Point assignments don't change**
    Even if the centroids move a little, if every point remains assigned to the same cluster as the previous iteration, the process can stop. 

3. **Maximum number of iterations reached**
    To avoid infinite loops, a maximum limit like 100 or 300 iterations is set. If the algorithm hasn't converged by then, it stops. 

Most implementations (like FAISS) use:
- **Tolerance**: If the centroid movement is smaller than a small threshold (say 10-4), stop. 
- **Max iterations**: Even if it is not fully converged, stop after N iterations. 

### Compressing Vectors

#### Product Quantization (PQ)
In similarity search, storing and comparing raw vectors is expense. If we consider every vector dimension to take 4 bytes of memory, 1 billion vectors with 128 dimensions would take 512 GB of RAM. We need a way to compress vectors so we can still compare them approximately but with much less memory and faster distance computation. That's where Product Quantization comes in 

##### The idea of Product Quantization
Instead of representing a vector by its raw float value, PQ **splits it into smaller chunks** and quantizes each chunk separately. 

- **Step 1: Split the vector**
    - Suppose we have a vector of dimensions _D_ = 128
    - Split it into _M_ sub-vectors. If we consider _M_ to be 8, each sub-vector will have _D/M_ = 16 dimensions.

- **Step 2: Quantize each sub-vector**
    - For each sub-vector space, run **k-means** to create a code-book (dictionary) of centroids. If you choose 256 centroids (k=256), then each sub-vector can be approximated by just 1 byte. (0-255 index)

- **Step 3: Encode the vector**
    - Instead of storing the raw 128 floats, we store the **index of the closest centroid for each sub-vector. For 8 sub-vector, that would be 8 bytes in total. 

So the whole 128-D float vector (~512 bytes) is compressed to just 8 bytes.

##### Distance Computation with PQ
How do we search efficiently if we don't have raw vectors?

1. When a **query vector** comes in, we split it into sub-vectors too. 
2. For each sub-vector, compute its distance to all centroids in the code-book. 
3. For each database vector, look up its stored centroid index in each subspace and sum up the distances. 

This way, distance computation is super fast. Just table lookups + additions, instead of full vector distance calculations. 

#### Scalar Quantization (SQ)
Scalar Quantization (SQ), instead of storing full-precision floats for each vector dimension, it approximates each component (dimension) by a discrete value - a "quantized" scaler. 

Each dimension is treated independently unlike in PQ which splits vectors into sub vectors. 

##### How it works?
- **Step 1: Choose the number of bits per dimension**
    An 8 Bit SQ can take 256 discrete values. A 4 bit SQ can take 16 discrete values.

- **Step 2: Find min/max values**
    For each dimension across all vectors, determine the minimum (Xmin) and maximum values (Xmax)

- **Step 3: Map each value to a discrete level**
    - Each dimension range (Xmin , Xmax) is divided into 2^b levels (where b is the number of bits)
    - Each value is replaced by the index of its nearest level. 

    **Example**
    - Suppose dimension values range (minimum and maximum) is [0,1]
    - 8-bit --> 256 levels: 0.0, 0.0039, 0.0078, ..., 0.996
    - A value of 0.45 --> nearest level 0.4492 --> store index 115 instead of the float. 

- **Step 4: Store compressed vectors**
    - Each dimension is replaced by its quantization index. 
    - 128-D vector with 8-bit SQ --> 128 bytes instead of 512 bytes (128 floats x 4 bytes).

#### Key differences in Quantization methods

| Feature | Product Quantization (PQ) | Scalar Quantization (SQ) |
|---------|---------------------------|--------------------------|
| Compression Method | Vectors split into sub-vectors | Vectors approximated by a scalar value 
| Complexity | Compute intensive | Simple - No k means per sub-vector |
| Memory Efficiency | Very high compression possible | Less efficient than PQ at same precision |
| Accuracy | Higher accuracy with the same code size | Less precise than PQ |
| Speed | Comparatively slower but still fast. Slow due to looks involved | Very fast |

### Indexes in FAISS
Index in FAISS decides how vectors are inserted, stored and retrieved. Every Index works with a similarity metric. (L2 distance , dot product etc)


1. Flat (Brute Force) Index

    - It stores all vectors in memory and compares query against all vectors. It supports euclidean distance ```IndexFlatL2``` and inner product ```IndexFlatIP``` (cosine similarity if the vectors are normalized)
    - Pros: Exact search, very simple
    - Cons: Slow for very large datasets (O(n) comparisons)
    - Use cases: Can be used when the dataset is small

2. Inverted File Index (IVF)

    Inverted File Index (IVF) divides the vector space into clusters using k-means. At search time, it only probes a subset of clusters instead of all vectors. It uses the below parameters to perform the search

    - nlist --> number of clusters
    - nprobe --> number of clusters searched per query 
    
    The different variants of Inverted File Index (IVF) are 

    - IVFFlat: Stores raw vectors in each cluster.
    - IVFPQ: Uses Product Quantization to compress vectors in each space. 
    - IVFSQ: Uses Scalar Quantization to compress vectors in each space. 

    - Pros: Much faster than brute force, scalable to billions of vectors
    - Cons: Approximate
    - Use case: Large datasets where speed is important. 

#### Flat Indexes (Exact Search)
Flat indexes just encode the vectors into codes of a fixed size and store them in an array of ntotal * code_size bytes. At search time, all the indexed vectors are decoded sequentially and compared to the query vectors. 

### Fast Nearest Neighbor Search
Nearest Neighbor Search looks for vectors closest to the query vector. In a large RAG applications, where we may have millions of document embeddings, a naive search would compute distances of the query to every single vector which would be very slow. 

Fast Nearest Neighbor Search uses optimized data-structures and algorithms. Fast Nearest Neighbor Search is a general term for any optimization that makes nearest neighbor retrieval quicker than brute force. The below optimization techniques are generally employed in Fast Nearest Neighbor Search

1. Hardware Level optimizations
2. Indexing Structures for Exact NNS
3. Approximate Nearest Neighbor (ANN) Structures

### FAISS

FAISS stands for Facebook AI Similarity Search. It is a library for fast nearest neighbor search in high-dimensional vector spaces. FAISS provides specialized data structures and algorithms to make Nearest Neighbor Search (NNS) faster. 

#### Core Features of FAISS
